# linear model to classify the MNIST data set

In this second tutorial, we will continue to work on image classification and try a linear classification model. This kind of model have the same number of parameters as the input images (64 here) plus one bias. They work by trying to with the parameters so that we minimize some Loss function at training time. At test time, a prediction is fast as it is basically just a dot product.

## Prepare and get a sense of the data

We start by loading our image data set: MNIST. Using the function `load_digits` of the `datasets` module of `sklearn` provide the dataset in a reduced form suitable for this practival session.

In [ ]:
# import numpy and initialize the random seed to yield consistent results
import numpy as np
np.random.seed(42)

In [ ]:
from sklearn.datasets import load_digits
mnist = ...

In [ ]:
mnist.keys()

The data set need to be partitioned into train and test data. Here use the handy function `train_test_split` of `sklearn` to reserve 20% of the data to test your model.

**/!\ The test data is to be left untouched.**

In [ ]:
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(..., test_size=...)

print('shape of train data is {}, type is {}'.format(X_train.shape, X_train.dtype))
print('shape of test data is {}, type is {}'.format(X_test.shape, X_test.dtype))

observe the data points: they are in 64 bits floats but only integers values from 0 to 16. The data can therefore be safely casted to uint8 to reduce the memory footprint by a factor of 8.

In [ ]:
print(...)  # min
print(...)  # max
print(...)  # unique
X_train = X_train.astype(...)

plot an image using matplotlib. The function `imshow` can be used reshaping the data as $(8\times8)$ array.

In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt, cm

index = 0
plt.imshow(...), cmap=cm.gray_r)
plt.axis('off')
plt.title('image %d in the train set' % index)

With this particular dataset, the list of the categories is identical to their indices (from 0 to 9).

Print the class of image `index`.

In [ ]:
print('image {} is a {}'.format(..., ...))

## Model definition

Here we define our simple machine learning algorithm which takes the features $x$, multiply them be some weights $W$ and add a bias term $b$.

$$f(x, W, b) = W.x + b = s$$

For a given image in vector form with $d$ features, W has size (10, d) so that the product $W.X$ produces 10 numbers which are called the scores for each class.

Initialize `numpy` arrays of size (10, 64) for $W$ and (10) for $b$. Concatenate $b$ and $W$ using the function `np.c_` to use the bias trick.

In [ ]:
# initialization with random weights
W = 0.1 * np.random.randn(...)
b = 0.1 * np.random.randn(...)

# apply the bias trick
W = ...
print('shape of W is now {}'.format(W.shape))

The data points are already in vector form, let's add 1 to each for the bias trick.

In [ ]:
X_train = np.c_[..., X_train]
print('shape of train data is now {}'.format(X_train.shape))

X_test = np.c_[..., X_test]
print('shape of test data is now {}'.format(X_test.shape))

now compute the 10 scores for the `index` training image with a dot product using `np.dot` and use the max score to determine the prediction

In [ ]:
scores = np.dot(...)
# look at the individual score for each class
for (label, score) in zip(labels, scores):
    print('{}: {:5.2f}'.format(..., ...))

Print the result, note that as we have 10 scores, we need to find the index of the maximum score to determine the class.

In [ ]:
print('prediction: {}'.format(...)
print('ground thruth: {}'.format(...)

## Loss function

### Hinge loss

We now need to define a way to tell the machine how happy we are with this prediction. The machine will then use this information to learn and come up with better predictions. The measure of our "happiness" is called a *loss function* and the process of learning the parameters (both $W$ and $b$) is called optimisation.

One possibility to measure how good is the prediction is the so called Hinge Loss:

$$L_i=\sum_{j\neq y^i}\max(0, s_j - s_{y^i} + 1)$$

Since it is inspired by linear support vector machines, this loss is also called Multi-class SVM Loss.

Now we can average arithmetically the losses $L_i$ for each instance $x^i$ to compute the general loss $L$ of the model.

$$L=\frac{1}{n}\sum_i L_i(f(x^i, W), y^i)$$

In [ ]:
# step by step calculation of the loss
Li = 0
yi = ...  # ground truth target
for j in range(...):
    if j == yi:
        print('skipping %d' % j)
        continue
    margin = ...
    print('{:2d} {:6.2f} {:6.2f}'.format(j, scores[j], margin))
    Li += ...
print(18 * '-')
print('hinge loss is {:.1f}'.format(Li))

Now we understand how the hinge loss works, we can use a more efficient implementation and include it in a reusable function.

Create a function (using `def`) called `loss_i` that compute the loss for given parameters `W` and `index`.

In [ ]:
# inline calculation of the loss
yi = np.squeeze(y_train)[index]
Li = np.sum([max(0, scores[j] - scores[yi] + 1) for j in range(10) if j != yi])
print(Li)

# create a function to evaluate the loss for the given W for image index in the training set
def loss_i(...):
    yi = ...  # ground truth target
    scores = ...
    Li = np.sum([max(0, scores[j] - scores[yi] + 1) for j in range(10) if j != yi])
    return Li

print(loss_i(W, index))

Finally create a function to compute the average loss on a batch of images

In [ ]:
def loss_batch(W, batch_size=100):
    L = 0.  # average loss
    for index in range(batch_size):
        L += ...
    L /= batch_size
    return L

In [ ]:
loss_batch(W, batch_size=50)

### Softmax loss

Another very popular loss function to use with multiclassification problems is the multinomial logistic or softmax loss (popular in deep learning). Here the score for each class is passed to the softmax function: exponentiated (and become positive) and normalized. This gives the probability distribution of this class:

$$P(Y=k|X=x_i)=\frac{e^{s_k}}{\sum_j e^{s_j}}$$

Now we have a probability we can try to maximize the likelihood which is equivalent to minimize the negative of the log likelihood:

$$L_i=-\log P(Y=k|X=x_i)=-\log\left(\frac{e^{s_k}}{\sum_j e^{s_j}}\right)$$

In [ ]:
# start by exponentiating our scores to obtain unnormalized probabilities
escores = np.exp(scores)
norm_escores = escores / np.sum(escores)
for j in range(10):
    print('{:6d} | {:8.1f} | {:6.4f}'.format(j, escores[j], norm_escores[j]))
print(26 * '-')
# verify that the sum of the probability is 1
print('sum of probabilities check: {:.3f}'.format(np.sum(norm_escores)))
# compute the softmax loss
Li = -np.log(norm_escores[yi])
print('Softmax loss is {:.2f}'.format(Li))

## Learning the model

Here we use the calculated loss to optimize the parameters $W$ and $b$. For this we need to evaluate the gradient $\dfrac{\partial L}{\partial W}$ of $L$ with respect to $W$.

The gradient is obtained by differentiating the loss expression with respect to $W$:

$$\nabla_{w_j}L_i=1\left(w_j^T x_i - w_{y_i}^T x_i + 1 > 0\right) x_i\quad\text{for }j\neq y_i$$

$$\nabla_{w_{y_i}}L_i=-\left(\sum_{j\neq y_i}1\left(w_j^T x_i - w_{y_i}^T x_i + 1 > 0\right)\right) x_i$$

with $1(condition)$ equals to 1 if $condition$ is true, 0 otherwise. Here we see that the data vector $x$ is scaled by the number of classes that did not meet the margins.

In [ ]:
# verify one more time the size of our matrices
print('shape of train data is {}'.format(X_train.shape))
print('shape of W is {}'.format(W.shape))

### Implementation

Simple SVM loss gradient implementation:
 - iterate over each data point $i$ in the batch
 - compute the score using $W.x^i$ (bias trick)
 - compute the margin for each class
 - compute the loss and the gradient components associated with this data point
 - finally average the gradient and the loss with respect to the number of data points in the batch

In [ ]:
def svm_loss_gradient(W, X, y):
    """
    SVM loss gradient.

    Inputs:
    - W: array of shape (K, 1 + D) containing the weights.
    - X: array of shape (N, 1 + D) containing the data.
    - y: array of shape (N, 1) containing training labels 0 <= k < K.
    Returns a tuple of:
    - average loss
    - gradient of the loss with respect to weights W
    """
    dW = np.zeros_like(W)  # initialize the gradient as zero
    K = ...  # number of classes
    n = ...  # number of data points
    loss = 0.0
    for i in range(n):
        #print('evaluating gradient / image %d' % i)
        yi = np.squeeze(y)[i]  # ground truth target
        scores = ...
        # compute SVM loss and gradient for this data point
        for j in range(K):
            if j == yi:
                continue
            # only compute loss if incorrectly classified
            margin = ...
            if margin > 0:
                loss += margin
                dW[yi, :] -= ...  # correct class gradient
                dW[j, :] += ...  # incorrect class gradient

    # average the loss and gradient
    loss /= n
    dW /= n
    return loss, dW

Now try our SVM gradient loss by computing the gradient with respect to the first `nb` images in the training set.

In [ ]:
nb = 100
loss, dW = svm_loss_gradient(...)
print('loss is {:.2f}'.format(loss))
print('gradient dW with respect to the first pixel =', dW[:, 2])

### Gradient check

now, to verify our SVM gradietn implementation, we are going to perform a **gradient check**. 

The gradient is computed numerically using a finite difference scheme:

$$\nabla L\approx\dfrac{L(W+h) - L(W-h)}{2h}$$

In [ ]:
def gradient_check(f, W, h=0.0001):
    dL = np.zeros_like(W)
    # evaluate the loss modifiying each value of W
    for c in range(W.shape[0]):
        for p in range(W.shape[1]):
            W[c, p] += h
            fxph = f(W)
            W[c, p] -= 2*h
            fxmh = f(W)
            dL[c, p] = ...  # centered finite differences
            W[c, p] += h  # put back initial value
    return dL

apply our gradient check, print the gradient with respect to the first pixel. Compare with the analytical value. Realize that to evaluate the gradient numerically, the loss function was called $2\times64$ times. This is why it is so slow. And we tested it only with 100 training images over 1437!

In [ ]:
print('loss is {:.2f}'.format(loss_batch(W, batch_size=100)))
dL = gradient_check(loss_batch, W)
print(dL.shape)

In [ ]:
print(dL[:, 2])

### Gradient Descent

now we have successfully created our linear model, loss function, and that we can compute the gradient of the loss with respect to $W$, let's actually use this to perform gradient descent and learn our model.

The backbone of the gradient descent is this simple equation:
$$W\leftarrow W - \eta \nabla_W L$$

$\eta$ is the learning rate (the most important hyperparameter). The weights $W$ are being updated at each iteration until a stop criterion is met or a maximum number of iteration reached.

In [ ]:
# examine one single gradient descent step
W = 0.1 * np.random.randn(10, 65)
print('average loss is %.1f' % loss_batch(W, batch_size=X_train.shape[0]))
loss, dL_dw = svm_loss_gradient(W, X_train, y_train)

# perform one gradient descent
eta = 0.005
W = W - eta * dL_dw
print('after one step the average loss is %.1f' % loss_batch(W, batch_size=X_train.shape[0]))

### Mini-batch gradient descent

because $n$ is large (1437 here, but can also be much much larger), it does not actually make sense of computing the gradient on the complete set of training images at each iteration (remeber that the gradient is averaged). Instead, it is very common to compute the gradient on a subset (called a mini-batch) of 32 to 256 images. This is much faster and performs well.

In [ ]:
W = np.random.randn(10, 65)  # initialization of the coefficients
eta = 0.005  # learning rate (< 1)
batch_size = 128
loss_history = []
it = 0
while it < 2000:
    # prepare batch
    idxs = np.random.choice(range(X_train.shape[0]), size=batch_size, replace=True)
    X_batch = X_train[idxs, :]
    y_batch = y_train[idxs]
    # evaluate loss and gradient
    loss, dL_dw = ...
    print('it {:d} - loss {:.1f}'.format(it, loss))
    # gradient descent
    W = ...
    loss_history.append(loss)
    it += 1

In [ ]:
plt.plot(loss_history)

Now make some prediction! Try the first 20 entries in the test set.

In [ ]:
for i in range(20):
    y_pred = ...
    print('{} - {}'.format(y_pred, y_test[i]))

Construct the confusion matrix which is usefull to measure the performances of our multinomial classifier.

In [ ]:
from sklearn.metrics import confusion_matrix

y_train_pred = ...
conf = confusion_matrix(...)

In [ ]:
plt.imshow(conf)
plt.xlabel('predicted class')
plt.ylabel('actual class')
plt.title('confusion matrix')

To better visualize the errors, it is useful to normalize each row by the total number of samples in each category.

In [ ]:
row_sums = conf.sum(axis=1, keepdims=True)
norm_conf = conf / row_sums

In [ ]:
np.fill_diagonal(norm_conf, 0)
plt.imshow(norm_conf)
plt.xlabel('predicted class')
plt.ylabel('actual class')
plt.title('matrix of error rates')

The columns for classes 8 and 9 look worse than the other. Analyzing the type of errors of the model can help improving it.

Finally we can compare our results with the `SGDClassifier` from `sklearn`.

## Compare our gradient descent results with sklearn

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(mnist['data'], mnist['target'], test_size=0.2)

In [ ]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(...)
for i in range(20):
    print('{} - {}'.format(y_pred[i], y_test[i]))

Compute the **accuracy** by dividing the number of correct prediction in the train set by the number os training samples.

In [ ]:
y_train_pred = clf.predict(...)
print(np.sum(...) / ...)

It is better to perform K-fold cross validation to measure the performances of the model. For this we can use the `cross_val_score` method with `cv=3`.

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(clf, X_train, y_train, cv=3)

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_train, y_train_pred)

In [ ]:
plt.imshow(conf)
plt.xlabel('predicted class')
plt.ylabel('actual class')
plt.title('confusion matrix')

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(251); plt.imshow(clf.coef_[0].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(252); plt.imshow(clf.coef_[1].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(253); plt.imshow(clf.coef_[2].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(254); plt.imshow(clf.coef_[3].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(255); plt.imshow(clf.coef_[4].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(256); plt.imshow(clf.coef_[5].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(257); plt.imshow(clf.coef_[6].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(258); plt.imshow(clf.coef_[7].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(259); plt.imshow(clf.coef_[8].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.subplot(2, 5, 10); plt.imshow(clf.coef_[9].reshape((8, 8)), cmap=cm.gray); plt.axis('off')
plt.show()